# Waymo Data Downloader
This notebook will download and store the Waymo image data into a GCP storage bucket, that can be used for supervised machine learning and processing.  In order to run the notebook, the Google Cloud CLI tools need to be installed.  For info on installation on your platform, visit:

https://cloud.google.com/sdk/docs/install-sdk

## Install Required Libraries

In [ ]:
%pip install -r requirements.txt

## Imports

In [ ]:
import autonomoeye
import google.auth
import tensorflow as tf
from tqdm import tqdm

## Google Authentication

In [ ]:
# Setup credentials
!gcloud auth login
!gcloud auth application-default login
!gcloud config set project capstone2023-378615
credentials, _ = google.auth.default()

## Load Missing Segment Info

In [ ]:
datatype = "validation"
annotation_uris = autonomoeye.get_annotation_uris(datatype)
uris = autonomoeye.get_all_waymo_segment_uris(datatype)
processed_segments = [autonomoeye.get_segment_name(uri) for uri in annotation_uris]
all_segments = [autonomoeye.get_segment_name(uri) for uri in uris]
missing_segments = [uri for uri in all_segments if uri not in processed_segments]

## Import Missing Segments

In [ ]:
for segment in (pbar:=tqdm(missing_segments)):
    pbar.set_postfix_str(segment)
    # Test loading the recordset
    uri = f"gs://waymo_open_dataset_v_1_4_1/individual_files/{datatype}/segment-{segment}_with_camera_labels.tfrecord"
    dataset = tf.data.TFRecordDataset(uri, compression_type='')

    # initialize annnotations dictionary
    annotations = autonomoeye.initialize_annotations_dict()

    # process images and annotations in frame and return metadata
    metadata = autonomoeye.process_segment(dataset, annotations, "waymo-processed-images", datatype, "tmp")